## 1. Load raw data

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
df_daily_rent = pd.read_csv("../data/raw/daily_rent_detail.csv")
df_weather = pd.read_csv("../data/raw/weather.csv")

/tmp/ipykernel_194094/1505384709.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_daily_rent = pd.read_csv("../data/raw/daily_rent_detail.csv")


## 2. Data Cleaning

In [34]:
df_daily_rent_copy = df_daily_rent.copy()
df_weather_copy = df_weather.copy()

In [35]:
df_daily_rent_copy.shape

(16086672, 13)

In [36]:
# Check duplicated ride_id 
check_duplicates = df_daily_rent_copy.duplicated(subset="ride_id").sum()
print("Duplicated ride_id amount:", check_duplicates)

# Drop duplicated ride_id 
df_daily_rent_copy = df_daily_rent_copy.drop_duplicates(subset="ride_id", keep='first')

Duplicated ride_id amount: 141


In [37]:
df_daily_rent_copy.shape

(16086531, 13)

In [ ]:
# 1. Calculate amount & percent
check_nan = df_daily_rent_copy.isna().sum()
check_nan_percent = df_daily_rent_copy.isna().mean() * 100

# 2. Merge to data frame
# axis=1 merge by col
nan_summary = pd.concat([check_nan, check_nan_percent], axis=1)

# 3. Set col name
nan_summary.columns = ["count", "percent"]

# 4. Display if nan amount > 0
display(nan_summary[nan_summary["count"] > 0])

,count,percent
start_station_name,1450130,9.014560
start_station_id,1450130,9.014560
end_station_name,1558227,9.686532
end_station_id,1558876,9.690567
start_lat,10,0.000062
start_lng,10,0.000062
end_lat,25916,0.161104
end_lng,25916,0.161104


In [46]:
# Drop nan
df_daily_rent_copy = df_daily_rent_copy.dropna()

In [47]:
df_daily_rent_copy.shape

(13928123, 13)

## 3. Data Integration

In [71]:
# Retrive necessary features from raw data
df_weather_needs = df_weather_copy[["datetime","icon"]].copy()
df_daily_rent_needs = df_daily_rent_copy[["started_at"]].copy()

# Create necessary data by merging from raw data 
df_temp = pd.to_datetime(df_daily_rent_needs["started_at"], format="mixed")
df_daily_rent_needs["datetime"] = df_temp.dt.strftime("%Y-%m-%d")
df_final = pd.merge(df_daily_rent_needs, df_weather_needs, how="left", on="datetime")

In [51]:
# Check if missing data after merging
assert df_daily_rent_copy.shape[0] == df_final.shape[0]

In [52]:
# Save to
df_final.to_csv("../data/raw/final.csv", index=False)

## 4. Feature engineering

In [63]:
df_raw = pd.read_csv("../data/raw/final.csv")

In [64]:
df = df_raw.copy()

In [ ]:
df_temp = pd.to_datetime(df["started_at"], format="mixed")

df["time"] = pd.DataFrame(df_temp.dt.strftime("%H:%M"))
df["dow"] =pd.DataFrame(df_temp.dt.day_name())

In [69]:
season_map = {
    1: 'winter', 2: 'spring', 3: 'spring', 4: 'spring',
    5: 'summer', 6: 'summer', 7: 'summer',
    8: 'autumn', 9: 'autumn', 10: 'autumn',
    11: 'winter', 12: 'winter'
}

df["season"] = df_temp.dt.month.map(season_map)

In [70]:
df.head()

,started_at,datetime,icon,time,dow,month,season
0,2020-05-30 17:25:29,2020-05-30,rain,17:25,Saturday,05,summer
1,2020-05-09 14:42:04,2020-05-09,partly-cloudy-day,14:42,Saturday,05,summer
2,2020-05-24 17:27:19,2020-05-24,cloudy,17:27,Sunday,05,summer
3,2020-05-27 15:29:52,2020-05-27,partly-cloudy-day,15:29,Wednesday,05,summer
4,2020-05-31 14:06:03,2020-05-31,partly-cloudy-day,14:06,Sunday,05,summer
